# How to review tool calls (Functional API)

!!! info "Prerequisites"
    This guide assumes familiarity with the following:

    - Implementing [human-in-the-loop](../../concepts/human_in_the_loop) workflows with [interrupt](../../concepts/human_in_the_loop/#interrupt)
    - [How to create a ReAct agent using the Functional API](../../how-tos/react-agent-from-scratch-functional)

This guide demonstrates how to implement human-in-the-loop workflows in a ReAct agent using the LangGraph [Functional API](../../concepts/functional_api).

We will build off of the agent created in the [How to create a ReAct agent using the Functional API](../../how-tos/react-agent-from-scratch-functional) guide.

Specifically, we will demonstrate how to review [tool calls](https://js.langchain.com/docs/concepts/tool_calling/) generated by a [chat model](https://js.langchain.com/docs/concepts/chat_models/) prior to their execution. This can be accomplished through use of the [interrupt](../../concepts/human_in_the_loop/#interrupt) function at key points in our application.

**Preview**:

We will implement a simple function that reviews tool calls generated from our chat model and call it from inside our application's [entrypoint](../../concepts/functional_api/#entrypoint):

```ts
function reviewToolCall(toolCall: ToolCall): ToolCall | ToolMessage {
  // Interrupt for human review
  const humanReview = interrupt({
    question: "Is this correct?",
    tool_call: toolCall,
  });

  const { action, data } = humanReview;

  if (action === "continue") {
    return toolCall;
  } else if (action === "update") {
    return {
      ...toolCall,
      args: data,
    };
  } else if (action === "feedback") {
    return new ToolMessage({
      content: data,
      name: toolCall.name,
      tool_call_id: toolCall.id,
    });
  }
  throw new Error(`Unsupported review action: ${action}`);
}
```

## Setup

!!! note Compatibility

    This guide requires `@langchain/langgraph>=0.2.42`.

First, install the required dependencies for this example:

```bash
npm install @langchain/langgraph @langchain/openai @langchain/core zod
```

Next, we need to set API keys for OpenAI (the LLM we will use):

```typescript
process.env.OPENAI_API_KEY = "YOUR_API_KEY";
```

!!! tip "Set up [LangSmith](https://smith.langchain.com) for LangGraph development"

    Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM apps built with LangGraph — read more about how to get started [here](https://docs.smith.langchain.com)

## Define model and tools

Let's first define the tools and model we will use for our example. As in the [ReAct agent guide](../../how-tos/react-agent-from-scratch-functional), we will use a single place-holder tool that gets a description of the weather for a location.

We will use an [OpenAI](https://js.langchain.com/docs/integrations/providers/openai/) chat model for this example, but any model [supporting tool-calling](https://js.langchain.com/docs/integrations/chat/) will suffice.

In [1]:
import { ChatOpenAI } from "@langchain/openai";
import { tool } from "@langchain/core/tools";
import { z } from "zod";

const model = new ChatOpenAI({
  model: "gpt-4o-mini",
});

const getWeather = tool(async ({ location }) => {
  // This is a placeholder for the actual implementation
  const lowercaseLocation = location.toLowerCase();
  if (lowercaseLocation.includes("sf") || lowercaseLocation.includes("san francisco")) {
    return "It's sunny!";
  } else if (lowercaseLocation.includes("boston")) {
    return "It's rainy!";
  } else {
    return `I am not sure what the weather is in ${location}`;
  }
}, {
  name: "getWeather",
  schema: z.object({
    location: z.string().describe("Location to get the weather for"),
  }),
  description: "Call to get the weather from a specific location.",
});

const tools = [getWeather];

## Define tasks

Our [tasks](../../concepts/functional_api/#task) are unchanged from the [ReAct agent guide](../../how-tos/react-agent-from-scratch-functional):

1. **Call model**: We want to query our chat model with a list of messages.
2. **Call tool**: If our model generates tool calls, we want to execute them.

In [2]:
import {
  type BaseMessageLike,
  AIMessage,
  ToolMessage,
} from "@langchain/core/messages";
import { type ToolCall } from "@langchain/core/messages/tool";
import { task } from "@langchain/langgraph";

const toolsByName = Object.fromEntries(tools.map((tool) => [tool.name, tool]));

const callModel = task("callModel", async (messages: BaseMessageLike[]) => {
  const response = await model.bindTools(tools).invoke(messages);
  return response;
});

const callTool = task(
  "callTool",
  async (toolCall: ToolCall): Promise<AIMessage> => {
    const tool = toolsByName[toolCall.name];
    const observation = await tool.invoke(toolCall.args);
    return new ToolMessage({ content: observation, tool_call_id: toolCall.id });
    // Can also pass toolCall directly into the tool to return a ToolMessage
    // return tool.invoke(toolCall);
  });

## Define entrypoint

To review tool calls before execution, we add a `reviewToolCalls` function that calls [interrupt](../../concepts/human_in_the_loop/#interrupt). When this function is called, execution will be paused until we issue a command to resume it.

Given a tool call, our function will `interrupt` for human review. At that point we can either:

- Accept the tool call;
- Revise the tool call and continue;
- Generate a custom tool message (e.g., instructing the model to re-format its tool call).

We will demonstrate these three cases in the [usage examples](#usage) below.

In [3]:
import { interrupt } from "@langchain/langgraph";

function reviewToolCall(toolCall: ToolCall): ToolCall | ToolMessage {
  // Interrupt for human review
  const humanReview = interrupt({
    question: "Is this correct?",
    tool_call: toolCall,
  });

  const { action, data } = humanReview;

  if (action === "continue") {
    return toolCall;
  } else if (action === "update") {
    return {
      ...toolCall,
      args: data,
    };
  } else if (action === "feedback") {
    return new ToolMessage({
      content: data,
      name: toolCall.name,
      tool_call_id: toolCall.id,
    });
  }
  throw new Error(`Unsupported review action: ${action}`);
}

We can now update our [entrypoint](../../concepts/functional_api/#entrypoint) to review the generated tool calls. If a tool call is accepted or revised, we execute in the same way as before. Otherwise, we just append the `ToolMessage` supplied by the human.

!!! tip

    The results of prior tasks — in this case the initial model call — are persisted, so that they are not run again following the `interrupt`.

In [4]:
import {
  MemorySaver,
  addMessages,
  entrypoint,
  getPreviousState,
} from "@langchain/langgraph";

const checkpointer = new MemorySaver();

const agent = entrypoint({
  checkpointer,
  name: "agent",
}, async (messages: BaseMessageLike[]) => {
  const previous = getPreviousState<BaseMessageLike[]>() ?? [];
  let currentMessages = addMessages(previous, messages);
  let llmResponse = await callModel(currentMessages);
  while (true) {
    if (!llmResponse.tool_calls?.length) {
      break;
    }
    // Review tool calls
    const toolResults: ToolMessage[] = [];
    const toolCalls: ToolCall[] = [];
    
    for (let i = 0; i < llmResponse.tool_calls.length; i++) {
      const review = await reviewToolCall(llmResponse.tool_calls[i]);
      if (review instanceof ToolMessage) {
        toolResults.push(review);
      } else { // is a validated tool call
        toolCalls.push(review);
        if (review !== llmResponse.tool_calls[i]) {
          llmResponse.tool_calls[i] = review;
        }
      }
    }
    // Execute remaining tool calls
    const remainingToolResults = await Promise.all(
      toolCalls.map((toolCall) => callTool(toolCall))
    );
    
    // Append to message list
    currentMessages = addMessages(
      currentMessages,
      [llmResponse, ...toolResults, ...remainingToolResults]
    );

    // Call model again
    llmResponse = await callModel(currentMessages);
  }
  // Generate final response
  currentMessages = addMessages(currentMessages, llmResponse);
  return entrypoint.final({
    value: llmResponse,
    save: currentMessages
  });
});

### Usage

Let's demonstrate some scenarios.

In [5]:
import { BaseMessage, isAIMessage } from "@langchain/core/messages";

const prettyPrintMessage = (message: BaseMessage) => {
  console.log("=".repeat(30), `${message.getType()} message`, "=".repeat(30));
  console.log(message.content);
  if (isAIMessage(message) && message.tool_calls?.length) {
    console.log(JSON.stringify(message.tool_calls, null, 2));
  }
}

const printStep = (step: Record<string, any>) => {
  if (step.__metadata__?.cached) {
    return;
  }
  for (const [taskName, result] of Object.entries(step)) {
    if (taskName === "agent") {
      continue; // just stream from tasks
    }
    
    console.log(`\n${taskName}:`);
    if (taskName === "__interrupt__" || taskName === "reviewToolCall") {
      console.log(JSON.stringify(result, null, 2));
    } else {
      prettyPrintMessage(result);
    }
  }
};

### Accept a tool call

To accept a tool call, we just indicate in the data we provide in the `Command` that the tool call should pass through.

In [6]:
const config = {
  configurable: {
    thread_id: "1"
  }
};

const userMessage = {
  role: "user",
  content: "What's the weather in san francisco?"
};
console.log(userMessage);

const stream = await agent.stream([userMessage], config);

for await (const step of stream) {
  printStep(step);
}

{ role: 'user', content: "What's the weather in san francisco?" }



callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_pe7ee3A4lOO4Llr2NcfRukyp"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco"
        },
        "type": "tool_call",
        "id": "call_pe7ee3A4lOO4Llr2NcfRukyp"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:dcee519a-80f5-5950-9e1c-e8bb85ed436f"
    ]
  }
]


In [7]:
import { Command } from "@langchain/langgraph";

// highlight-next-line
const humanInput = new Command({
  // highlight-next-line
  resume: {
    // highlight-next-line
    action: "continue",
    // highlight-next-line
  },
  // highlight-next-line
});

const resumedStream = await agent.stream(humanInput, config)

for await (const step of resumedStream) {
  printStep(step);
}


callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco is sunny!


### Revise a tool call

To revise a tool call, we can supply updated arguments.

In [8]:
const config2 = {
  configurable: {
    thread_id: "2"
  }
};

const userMessage2 = {
  role: "user",
  content: "What's the weather in san francisco?"
};

console.log(userMessage2);

const stream2 = await agent.stream([userMessage2], config2);

for await (const step of stream2) {
  printStep(step);
}

{ role: 'user', content: "What's the weather in san francisco?" }

callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_JEOqaUEvYJ4pzMtVyCQa6H2H"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco"
        },
        "type": "tool_call",
        "id": "call_JEOqaUEvYJ4pzMtVyCQa6H2H"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:d5c54c67-483a-589a-a1e7-2a8465b3ef13"
    ]
  }
]


In [9]:
// highlight-next-line
const humanInput2 = new Command({
  // highlight-next-line
  resume: {
    // highlight-next-line
    action: "update",
    // highlight-next-line
    data: { location: "SF, CA" },
    // highlight-next-line
  },
  // highlight-next-line
});

const resumedStream2 = await agent.stream(humanInput2, config2)

for await (const step of resumedStream2) {
  printStep(step);
}


callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco is sunny!


The LangSmith traces for this run are particularly informative:

- In the trace [before the interrupt](https://smith.langchain.com/public/abf80a16-3e15-484b-bbbb-23017593bd39/r), we generate a tool call for location `"San Francisco"`.
- In the trace [after resuming](https://smith.langchain.com/public/233a7e32-a43e-4939-9c04-96fd4254ce65/r), we see that the tool call in the message has been updated to `"SF, CA"`.

### Generate a custom ToolMessage

To Generate a custom `ToolMessage`, we supply the content of the message. In this case we will ask the model to reformat its tool call.

In [10]:
const config3 = {
  configurable: {
    thread_id: "3"
  }
};

const userMessage3 = {
  role: "user",
  content: "What's the weather in san francisco?"
};

console.log(userMessage3);

const stream3 = await agent.stream([userMessage3], config3);

for await (const step of stream3) {
  printStep(step);
}

{ role: 'user', content: "What's the weather in san francisco?" }

callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco"
    },
    "type": "tool_call",
    "id": "call_HNRjJLJo4U78dtk0uJ9YZF6V"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco"
        },
        "type": "tool_call",
        "id": "call_HNRjJLJo4U78dtk0uJ9YZF6V"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:6f313de8-c19e-5c3e-bdff-f90cdd68d0de"
    ]
  }
]


In [11]:
// highlight-next-line
const humanInput3 = new Command({
  // highlight-next-line
  resume: {
    // highlight-next-line
    action: "feedback",
    // highlight-next-line
    data: "Please format as <City>, <State>.",
    // highlight-next-line
  },
  // highlight-next-line
});

const resumedStream3 = await agent.stream(humanInput3, config3)

for await (const step of resumedStream3) {
  printStep(step);
}


callModel:
============================== ai message ==============================

[
  {
    "name": "getWeather",
    "args": {
      "location": "San Francisco, CA"
    },
    "type": "tool_call",
    "id": "call_5V4Oj4JV2DVfeteM4Aaf2ieD"
  }
]

__interrupt__:
[
  {
    "value": {
      "question": "Is this correct?",
      "tool_call": {
        "name": "getWeather",
        "args": {
          "location": "San Francisco, CA"
        },
        "type": "tool_call",
        "id": "call_5V4Oj4JV2DVfeteM4Aaf2ieD"
      }
    },
    "when": "during",
    "resumable": true,
    "ns": [
      "agent:6f313de8-c19e-5c3e-bdff-f90cdd68d0de"
    ]
  }
]


Once it is re-formatted, we can accept it:

In [12]:
// highlight-next-line
const continueCommand = new Command({
  // highlight-next-line
  resume: {
    // highlight-next-line
    action: "continue",
    // highlight-next-line
  },
  // highlight-next-line
});

const continueStream = await agent.stream(continueCommand, config3)

for await (const step of continueStream) {
  printStep(step);
}


callTool:
============================== tool message ==============================
It's sunny!

callModel:
============================== ai message ==============================
The weather in San Francisco, CA is sunny!
